In [ ]:
##  一个售前和售后的 langchain  LLMRouterChain 模版

from langchain.chains.router import MultiPromptChain
from langchain_community.llms import Tongyi
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import (
    LLMRouterChain,
    RouterOutputParser
)
from langchain.chains.router.multi_prompt_prompt import (
    MULTI_PROMPT_ROUTER_TEMPLATE
)

# 售前咨询模板
presales_prompt_tpl = PromptTemplate.from_template(
    '你是一位专业的售前顾问，擅长产品介绍、方案推荐和商务咨询。'
    '你需要热情、专业地回答客户的产品咨询、价格询问、功能介绍等售前问题。'
    '请使用中文帮我解答下列售前咨询问题：\n{input}'
)

# 售后服务模板
aftersales_prompt_tpl = PromptTemplate.from_template(
    '你是一位耐心的售后服务专员，擅长解决客户的使用问题、技术支持和投诉处理。'
    '你需要耐心、细致地帮助客户解决产品使用中遇到的问题，提供技术支持和服务指导。'
    '请使用中文帮我解答下列售后服务问题：\n{input}'
)

# 创建模板信息列表
prompt_infos = [
    {
        'name': 'presales',
        'description': '用于处理售前咨询，包括产品介绍、价格询问、功能说明、方案推荐等',
        'prompt_template': presales_prompt_tpl,
    },
    {
        'name': 'aftersales',
        'description': '用于处理售后服务，包括使用问题、技术支持、故障排除、投诉处理等',
        'prompt_template': aftersales_prompt_tpl,
    },
]

llm = Tongyi(
    temperature=0.1,
)

# 生成键为模板名称、值为Chain的字典
destination_chains = {}
for p_info in prompt_infos:
    name = p_info['name']
    prompt = p_info['prompt_template']
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# 将模板名称和模板描述通过MULTI_PROMPT_ROUTER_TEMPLATE生成模板
destinations = [f'{p["name"]}: {p["description"]}'
                for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 这里创建了一个default_chain
# 为了防止提的问题类型并没有包含在prompt_infos中
default_chain = ConversationChain(llm=llm, output_key='text')
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

# 测试售前咨询问题
print("=== 售前咨询测试 ===")
print(chain.run("你们的产品有什么功能？价格是多少？"))

print("\n=== 售后服务测试 ===")
print(chain.run("我的产品出现故障了，无法正常启动，该怎么办？"))

print("\n=== 其他问题测试 ===")
print(chain.run("今天天气怎么样？"))


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21616\30894139.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21616\30894139.py:71: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  default_chain = ConversationChain(llm=llm, output_key='text')
d:\software\miniconda3\envs\MLOps\Lib\site-packages\pydantic\main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21616\30894139.py:72: LangC

=== 售前咨询测试 ===


> Entering new MultiPromptChain chain...
presales: {'input': '你们的产品有什么功能？价格是多少？'}
> Finished chain.
您好，感谢您对我们产品的关注！为了更好地为您介绍产品功能和价格信息，能否请您先告知您感兴趣的是哪一类产品或具体的产品名称呢？

我们的产品线覆盖多个领域，包括但不限于：

- 企业级软件解决方案（如CRM、ERP、OA系统等）  
- 网络与安全设备  
- 数据中心与云计算服务  
- 智能办公设备  
- 行业定制化解决方案（如教育、医疗、制造等）

不同产品的功能和价格体系差异较大，因此我需要了解您更具体的需求，以便为您提供准确的产品介绍、功能说明及报价方案。

期待您的回复，我将第一时间为您服务！

=== 售后服务测试 ===


> Entering new MultiPromptChain chain...
aftersales: {'input': '我的产品出现故障了，无法正常启动，该怎么办？'}
> Finished chain.
您好，非常抱歉听到您的产品出现了无法正常启动的问题。我是您的售后服务专员，将耐心为您解答并提供帮助。为了更好地判断问题原因并找到解决方案，请您配合我完成以下几步排查：

---

### 一、初步检查（请您逐一确认）：

1. **电源连接是否正常？**
   - 请确认电源线是否插好，插座是否有电（可以尝试插其他电器测试插座）。
   - 如果是使用电池供电的产品，请确认电池是否安装正确、电量是否充足。

2. **开关是否正常开启？**
   - 检查产品开关是否处于“ON”或启动位置，部分设备可能有多个开关（例如主机和显示屏分别有开关）。

3. **是否有指示灯或声音提示？**
   - 产品通电后是否有任何指示灯亮起、风扇转动声或提示音？这些信息对我们判断问题非常关键。

4. **是否有异味、异响或异常发热？**
   - 如果有烧焦味、冒烟、明显发热等现象，请**立即断电**，并避免继续使用，可能存在安全隐患。

---

### 二、尝试以下操作：

- **重启操作：**
  - 拔掉电源线或取出电池，等待10秒后重新连接，尝试再次启动。

- **检查配件或连接线：**

In [1]:
# 整合链的语法

from langchain.chains.router import MultiPromptChain
from langchain_community.llms import Tongyi
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import (
    LLMRouterChain,
    RouterOutputParser
)
from langchain.chains.router.multi_prompt_prompt import (
    MULTI_PROMPT_ROUTER_TEMPLATE
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 初始化LLM
llm = Tongyi(temperature=0.1)

# 售前咨询链 - 使用新式语法
presales_prompt = PromptTemplate.from_template(
    '你是一位专业的售前顾问，擅长产品介绍、方案推荐和商务咨询。'
    '你需要热情、专业地回答客户的产品咨询、价格询问、功能介绍等售前问题。'
    '请使用中文帮我解答下列售前咨询问题：\n{input}'
)
presales_chain = presales_prompt | llm | StrOutputParser()

# 售后服务链 - 使用新式语法
aftersales_prompt = PromptTemplate.from_template(
    '你是一位耐心的售后服务专员，擅长解决客户的使用问题、技术支持和投诉处理。'
    '你需要耐心、细致地帮助客户解决产品使用中遇到的问题，提供技术支持和服务指导。'
    '请使用中文帮我解答下列售后服务问题：\n{input}'
)
aftersales_chain = aftersales_prompt | llm | StrOutputParser()

# 意图识别链 - 使用新式语法
intent_prompt = PromptTemplate.from_template(
    """请分析以下用户问题的意图，判断是售前咨询还是售后服务：

售前咨询：产品介绍、功能说明、价格询问、方案推荐、购买咨询等
售后服务：使用问题、技术支持、故障排除、投诉处理、维修服务等

用户问题：{input}

请只回答"售前"或"售后"，不要添加其他内容。"""
)
intent_chain = intent_prompt | llm | StrOutputParser()

# 创建路由函数
def route_question(input_dict):
    question = input_dict["input"]
    intent = intent_chain.invoke({"input": question})
    
    print(f"识别意图: {intent.strip()}")
    
    if "售前" in intent:
        return presales_chain.invoke({"input": question})
    elif "售后" in intent:
        return aftersales_chain.invoke({"input": question})
    else:
        # 默认处理
        default_prompt = PromptTemplate.from_template(
            "我是一个智能助手，很高兴为您服务。请问有什么可以帮助您的吗？\n问题：{input}"
        )
        default_chain = default_prompt | llm | StrOutputParser()
        return default_chain.invoke({"input": question})

# 创建完整的路由链
router_chain = RunnablePassthrough() | RunnableLambda(route_question)

# 方法二：更简洁的条件路由实现
from langchain_core.runnables import RunnableBranch

# 创建条件判断函数
def is_presales(input_dict):
    intent = intent_chain.invoke(input_dict)
    return "售前" in intent

def is_aftersales(input_dict):
    intent = intent_chain.invoke(input_dict)
    return "售后" in intent

# 使用 RunnableBranch 创建条件路由
branch_chain = RunnableBranch(
    (is_presales, presales_chain),
    (is_aftersales, aftersales_chain),
    # 默认链
    PromptTemplate.from_template("我是智能助手，请问有什么可以帮助您的？\n{input}") | llm | StrOutputParser()
)

# 测试代码
if __name__ == "__main__":
    print("=== 方法一：自定义路由函数 ===")
    
    # 测试售前问题
    print("\n--- 售前咨询测试 ---")
    result1 = router_chain.invoke({"input": "你们的产品有什么功能？价格是多少？"})
    print(f"回答: {result1}")
    
    # 测试售后问题
    print("\n--- 售后服务测试 ---")
    result2 = router_chain.invoke({"input": "我的产品出现故障了，无法正常启动，该怎么办？"})
    print(f"回答: {result2}")
    
    print("\n=== 方法二：RunnableBranch 条件路由 ===")
    
    # 测试售前问题
    print("\n--- 售前咨询测试 ---")
    result3 = branch_chain.invoke({"input": "我想了解一下你们的服务套餐和收费标准"})
    print(f"回答: {result3}")
    
    # 测试售后问题
    print("\n--- 售后服务测试 ---")
    result4 = branch_chain.invoke({"input": "产品使用过程中遇到了错误提示，需要技术支持"})
    print(f"回答: {result4}")


=== 方法一：自定义路由函数 ===

--- 售前咨询测试 ---
识别意图: 售前
回答: 您好！感谢您对我们产品的关注。我将为您详细介绍我们的产品功能及价格相关信息。

一、产品核心功能介绍：

我们的产品是一套集成化、智能化的解决方案，主要功能包括：

1. **高效数据管理**  
   - 支持多源数据接入与整合  
   - 提供数据清洗、存储、分析全流程管理  
   - 实时数据可视化展示

2. **智能分析与决策支持**  
   - 内置AI算法模型，支持预测分析、趋势判断  
   - 自定义报表生成与多维度数据分析  
   - 智能预警与异常检测功能

3. **灵活部署与扩展**  
   - 支持本地部署、云端部署或混合部署模式  
   - 可根据企业需求进行模块化扩展  
   - 与主流系统（如ERP、CRM）无缝集成

4. **安全与权限管理**  
   - 多级权限控制，保障数据安全  
   - 完善的日志记录与审计机制  
   - 支持数据加密与灾备恢复

5. **用户友好体验**  
   - 简洁直观的操作界面  
   - 支持多终端访问（PC、移动端）  
   - 提供在线帮助与操作引导

二、产品价格说明：

我们的产品采用**模块化定价+订阅制/永久授权**的灵活收费模式，具体价格根据以下因素确定：

- 所选功能模块（基础版 / 专业版 / 定制版）  
- 用户数量与并发访问需求  
- 部署方式（云端 / 本地）  
- 是否需要定制开发与系统集成  
- 服务支持等级（标准支持 / 金牌服务）

为方便您了解，以下为大致的价格区间（具体以实际报价为准）：

| 产品版本     | 适用对象       | 起始价格（年费/授权） |
|--------------|----------------|------------------------|
| 基础版       | 中小型企业     | ￥50,000 起           |
| 专业版       | 中大型企业     | ￥150,000 起          |
| 定制版       | 大型企业/集团  | ￥300,000 起          |

我们还提供**免费试用**、**POC验证**和**定

## EmbeddingRouterChain

不仅可以使用 LLMRouteChain 来智能选择合适的处理链，还可以采用 EmbeddingRouterChain，该组件通过计算各 Chain 描述与用户问题之间的语义相关性，实现更精准的路由决策。


In [7]:
!pip install chromadb

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.5.4-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/19.8 MB ? eta -:--:--
   ----------------------------------------

In [ ]:
from langchain_community.vectorstores import Chroma    # # pip install chroma
from langchain_community.embeddings import DashScopeEmbeddings # pip install dashscope
from langchain.chains import LLMRouterChain, MultiPromptChain
from langchain_core.language_models import BaseLLM
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Tongyi  # 或你使用的 LLM
import os

# 1. 定义任务名称与描述
names_and_descriptions = [
    ("physics", ["用于解答物理相关问题，例如力学、电磁学等"]),
    ("math", ["用于解答数学相关问题，例如代数、几何、微积分等"]),
]

# 2. 使用通义千问的 Embedding 模型
embeddings = DashScopeEmbeddings(model="text-embedding-v2")

# 3. 构建向量数据库（用于路由匹配）
descriptions = []
names = []
for name, desc_list in names_and_descriptions:
    for desc in desc_list:
        descriptions.append(desc)
        names.append(name)

# 创建 Chroma 向量库
vectorstore = Chroma(embedding_function=embeddings)
# 批量添加文档
vectorstore.add_texts(texts=descriptions, metadatas=[{"name": name} for name in names])

# 4. 自定义 Embedding 路由链（LangChain 没有直接 from_names_and_descriptions）
def get_relevant_chain_name(question: str) -> str:
    docs = vectorstore.similarity_search(question, k=1)
    return docs[0].metadata["name"]

# 5. 定义各个目标链的 prompt 和 LLM
llm = Tongyi(model_name="qwen-plus", temperature=0.1)  # 可替换为你用的 LLM

physics_prompt = PromptTemplate(
    template="你是一个物理专家，请回答以下问题：\n{input}",
    input_variables=["input"]
)
math_prompt = PromptTemplate(
    template="你是一个数学专家，请回答以下问题：\n{input}",
    input_variables=["input"]
)

destination_chains = {
    "physics": physics_prompt | llm,
    "math": math_prompt | llm,
}

default_chain = PromptTemplate.from_template("请回答以下问题：{input}") | llm

# 6. 定义运行逻辑（模拟 MultiPromptChain）
def run_router_chain(question: str):
    chain_name = get_relevant_chain_name(question)
    print(f"路由到: {chain_name}")
    if chain_name in destination_chains:
        return destination_chains[chain_name].invoke({"input": question})
    else:
        return default_chain.invoke({"input": question})

# 7. 测试
result = run_router_chain("牛顿第一定律是什么？")
print(result)

路由到: physics
牛顿第一定律，也称为**惯性定律**，其内容是：

> **任何物体在不受外力作用时，总保持静止状态或匀速直线运动状态。**

换句话说，如果一个物体没有受到**净外力**（即合外力为零）的作用，那么它的**速度**将保持不变。这意味着：

- 原来静止的物体将继续保持静止；
- 原来运动的物体将以恒定速度沿直线运动。

---

### 补充说明：

- 牛顿第一定律最早由伽利略提出思想基础，后来由牛顿系统地总结为经典力学的第一条定律。
- 这条定律定义了**惯性参考系**的概念：在这样的参考系中，物体在没有受力时能保持匀速直线运动或静止。
- 定律强调了**惯性**的概念：物体具有保持当前运动状态的倾向。

---

### 举例说明：

- 当你在一辆快速行驶的汽车中突然刹车时，你的身体会向前倾。这是因为你的身体“想要”保持原来的运动状态（惯性），而车已经减速。
- 在太空中，如果一个物体以一定速度运动且不受引力或其他力作用，它会一直以这个速度沿直线运动。

---

### 总结公式表达（虽然牛顿第一定律本身没有直接公式，但可表示为）：

若  
$$
\sum \vec{F} = 0
$$  
则  
$$
\vec{v} = \text{常量}
$$

即：当合外力为零时，速度矢量保持不变。
